In [82]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from openai import OpenAI
from dotenv import load_dotenv
import os

In [83]:
load_dotenv(override=True)
my_api_key =os.getenv("OPENAI_API_KEY")
#print (f'key is {my_api_key}')

In [84]:
class LibraryState(TypedDict):
        question: str
        faq_answer: str
        checkout_info: str
        final_answer: str

In [85]:
client = OpenAI(api_key=my_api_key)

In [86]:
def ClassifierAgent(state: LibraryState):
    print ("ClassifierAgent ran")
    print(f"state:{state}")

    # Build the LLM Message
    message_to_llm = [
        {"role":"system", "content": ''' You are a classifier agent in a library system. Decide if the user is asking 
         about book availability/checkout or about library FAQs. Reply with JSON containing keys: faq_answer and checkout_info.'''},
        {"role":"user", "content": f"Question: {state['question']}"}
    ]

    # Call the OpenAI model
    response = client.chat.completions.create(
        model="gpt-5-mini",
        messages=message_to_llm,
        temperature=1,   # keep it deterministic for classification
        #max_tokens=150,
    )

    print (response)
    # Extract the content from the response
    answer = response.choices[0].message.content
    # Ideally, parse as JSON — here assuming model returns a dict-like string
    try:
        import json
        parsed = json.loads(answer)
        return {
            "faq_answer": parsed.get("fag_answer",""),
            "checkout_info": parsed.get("checkout_info","")
        }
    except Exception:
        # fallback if LLM gives plain text
        return {"faq_answer": answer, "checkout_info": ""}
    
    #@traceble 



def FAQAgent(state: LibraryState):
    print("FAQAgent ran")
    print(f"FAQAgent state", state)
    
    # return {}
    return {"faq_answer": "The library opens at 9:00 AM"}

def CheckoutAgent(state: LibraryState):
    print("CheckoutAgent ran")
    
    # return {}
    return {"checkout_info": "The Hobbit is available for checkout."}

def ResponseAgent(state: LibraryState):
    print("ResponseAgent ran")
    
    #return {"final_answer": "RESPONSE AGENT FINAL ANSWER"}
    return {"final_answer": f"FAQ: {state['faq_answer']} | Checkout: {state['checkout_info']}"}


In [87]:
# --- Build the Graph ---
builder = StateGraph(LibraryState)
builder.add_node("ClassifierAgent", ClassifierAgent)
builder.add_node("FAQAgent", FAQAgent)
builder.add_node("CheckoutAgent", CheckoutAgent)
builder.add_node("ResponseAgent", ResponseAgent)

builder.add_edge(START, "ClassifierAgent")
builder.add_edge("ClassifierAgent", "FAQAgent")
builder.add_edge("ClassifierAgent", "CheckoutAgent")
builder.add_edge("FAQAgent", "ResponseAgent")
builder.add_edge("CheckoutAgent", "ResponseAgent")
builder.add_edge("ResponseAgent", END)

graph = builder.compile()

result = graph.invoke({"question": "When does library open?"})
print("\n--Final Answer--")
print(result["final_answer"])

ClassifierAgent ran
state:{'question': 'When does library open?'}
ChatCompletion(id='chatcmpl-CDEsljtItSJg8ewdb9aGbG21OHs4h', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "faq_answer": "Do you mean a specific branch? I don\'t know which library you mean. Please tell me the branch or location so I can give exact hours. (Typical public-library hours are often Mon–Fri 9:00 AM–6:00 PM, Sat 9:00 AM–1:00 PM, Sun closed; check the library\'s website or call the branch to confirm.)",\n  "checkout_info": null\n}', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1757271507, model='gpt-5-mini-2025-08-07', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=486, prompt_tokens=57, total_tokens=543, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=384, rej

In [88]:
# --- Visualize ---
#pip install grandalf

#print(graph.get_graph().draw_ascii())
# graph.get_graph().draw_png("images/agentic_ai_library.png")
# print("Graph saved as agentic_ai_library.png")